In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
path = '/content/drive/Shareddrives/통분방+공모전/데이터/'
out_market = pd.read_csv(path+"카카오데이터.csv", encoding="euc-kr")
out_market.head()

,시장명,편의점 수,대형마트 여부,지하철역 여부,카페 수,음식점 수,학교 수,유동인구
0,(유)문경시중앙시장,17,0,0,75,489,6,949
1,(주)서시장,20,0,0,121,730,8,964
2,(주)신정시장,75,1,0,171,1112,8,3031
3,(주)야음시장,30,0,0,94,458,9,5566
4,(주)여수수산시장,29,0,0,162,913,7,3080


In [3]:
out_market[out_market.시장명=="공릉동도깨비시장"]

,시장명,편의점 수,대형마트 여부,지하철역 여부,카페 수,음식점 수,학교 수,유동인구
89,공릉동도깨비시장,83,1,1,287,1372,9,12038


In [4]:
out_market[out_market.시장명=="상계중앙시장"]

,시장명,편의점 수,대형마트 여부,지하철역 여부,카페 수,음식점 수,학교 수,유동인구
607,상계중앙시장,81,0,1,301,1706,14,7884


In [5]:
out_market.shape

(1401, 8)

In [6]:
out_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1401 entries, 0 to 1400
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   시장명      1401 non-null   object
 1   편의점 수    1401 non-null   int64 
 2   대형마트 여부  1401 non-null   int64 
 3   지하철역 여부  1401 non-null   int64 
 4   카페 수     1401 non-null   int64 
 5   음식점 수    1401 non-null   int64 
 6   학교 수     1401 non-null   int64 
 7   유동인구     1401 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 87.7+ KB


In [7]:
!pip install prince

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from sklearn.preprocessing import MinMaxScaler # 전체 데이터로 scaling했을 때 0, 1로 구성된 범주형 변수가 영향을 받지 않도록 함.
import numpy as np

# scaling
X = out_market.drop('시장명', axis=1)
X_std = pd.DataFrame(MinMaxScaler().fit_transform(X), columns = X.columns)
X_std["대형마트 여부"] = np.where(X_std["대형마트 여부"]==1, "Y", "N")
X_std["지하철역 여부"] = np.where(X_std["지하철역 여부"]==1, "Y", "N")

In [9]:
X_std.head()

,편의점 수,대형마트 여부,지하철역 여부,카페 수,음식점 수,학교 수,유동인구
0,0.097143,N,N,0.048294,0.092472,0.428571,0.022215
1,0.114286,N,N,0.077914,0.138517,0.571429,0.022685
2,0.428571,Y,N,0.110109,0.211502,0.571429,0.087540
3,0.171429,N,N,0.060528,0.086549,0.642857,0.167080
4,0.165714,N,N,0.104314,0.173481,0.500000,0.089078


In [ ]:
import prince

famd = prince.FAMD(n_components=2, n_iter=3, random_state=42).fit(X_std)
out = famd.transform(X_std)
out.head() # transformed data

,0,1
0,-0.407310,-1.486206
1,-0.194410,-1.419556
2,-0.471850,0.485510
3,-0.087227,-1.373825
4,-0.026269,-1.358626


In [ ]:
out.iloc[[89, 607], :] # 노원구 시장

,0,1
89,1.198664,1.413762
607,2.191095,-0.272275


In [ ]:
famd.explained_inertia_  # explained variance

array([0.54030418, 0.21842546])

In [ ]:
famd.V_ # 여기 변수 순서를 알아내야돼 : 이건 R이랑 같이 비교해보면 될 것 같기도

array([[-0.40269559,  0.44249505,  0.31494916, -0.28365796,  0.36021267,
         0.3242646 ,  0.34642808,  0.29313096,  0.14676753],
       [-0.25835225,  0.28388588, -0.66587968,  0.59972242,  0.12005666,
         0.12269365,  0.09595428,  0.08572662,  0.05980451]])

In [ ]:
X_std.columns

Index(['편의점 수', '대형마트 여부', '지하철역 여부', '카페 수', '음식점 수', '학교 수', '유동인구'], dtype='object')

In [ ]:
famd.column_correlations(X_std) # correlation between the original variables and the components

,0,1
대형마트 여부_N,0.433715,-0.167413
대형마트 여부_Y,-0.433715,0.167413
유동인구,0.366150,0.331087
음식점 수,0.904104,0.756164
지하철역 여부_N,-0.729583,-0.685972
지하철역 여부_Y,0.729583,0.685972
카페 수,0.855302,0.745797
편의점 수,0.938246,0.800926
학교 수,0.841775,0.705654
